In [ ]:
# 1) Find reliable news with correspondance to stock prices
# 2) Scrape news headlines+subheadlines
# 3) Pre-processing and data cleaning
# 4) Create/import dictionary
# 5) Break news headlines/subheadlines into tokens
# 6) Analyze + calculate score

In [ ]:
from GoogleNews import GoogleNews
import datetime

now = datetime.datetime.now().date()
def normalize_date(date):
    normalize = str(date).split('-')
    for i in normalize:
        normalize = normalize[::-1]
    normalize[1],normalize[0] = normalize[0],normalize[1]
    new_date = '/'.join(normalize)
    return str(new_date)


# mm/dd/yy format
googlenews = GoogleNews()
googlenews.setlang('en')

# Although increasing the timerange would seem logical since more data would mean a better fit, the overall sentiment became neutralised and did not provide meaningful results. Therefore, instead of having a large timerange, a shorter timerange (with less data as a tradeoff) was chosen to reflect the recent/relevant news only.
# googlenews.setTimeRange('07/28/2020', normalize_date(now))

In [ ]:
def search_news(company,search_date):
    googlenews.setTimeRange(search_date, search_date)
    googlenews.clear()
    selected_companies_list = []
    googlenews.search(str(company))
    #change range for custom range. But don't include 1, because it is already included
    #in the default option, and including it will duplicate results.

    # for i in range(2,4):
    #     googlenews.getpage(i)
    googlenews.getpage()

    # googlenews.getpage(2)
    # googlenews.getpage(3)
    # return googlenews.result()
    for i in googlenews.result():
        # filtered for Bloomberg and Yonhap News based on my intuition and experience 
        # realted to their reliabilities and accuracies.
        # BELOW COMMENTS MADE AFTER RUNNING TRIALS
        # after running the trials, the scores actually represented the stock price              trend when other medias were included. Limiting media to Bloomberg and                 Yonhap seems to provide better results for positive scores, and including              other including other medias seems to provide better results for negative              scores.
        # if ((i.get('media') == 'Bloomberg') or (i.get('media') == 'Yonhap News')):
           selected_companies_list.append(i)
    return selected_companies_list

# print(len(search_news('SK Innovation Bloomberg')))

import re
import pandas as pd

stock_name = 'SK Innovation'
stock_id = '096770'

# search_date = normalize_date(now)
daterange = pd.date_range('07/01/2020', normalize_date(now))


def gather_news(stock_name,search_date):
    news_no_filter = search_news(stock_name,search_date)
    news_yonhap = search_news(stock_name+' Yonhap',search_date)
    news_bloomberg = search_news(stock_name+' Bloomberg',search_date)
    news_korea = search_news('South Korea Bloomberg',search_date)

    combined_dates = []
    combined_headlines = []

    for i in news_no_filter:
        # print(i.get('title'))
        # remove brackets with irrelevant texts in title
        new = re.sub(r'\([^)]*\) ', '', i.get('title'))
        combined_headlines.append(new)
        new = search_date
        combined_dates.append(new)

    for i in news_yonhap:
        # print(i.get('title'))
        # remove brackets with irrelevant texts in title
        new = re.sub(r'\([^)]*\) ', '', i.get('title'))
        combined_headlines.append(new)

        combined_dates.append(search_date)

    for i in news_bloomberg:
        new = re.sub(r'\([^)]*\) ', '', i.get('title'))
        combined_headlines.append(new)

        combined_dates.append(search_date)

    for i in news_korea:
        new = re.sub(r'\([^)]*\) ', '', i.get('title'))
        combined_headlines.append(new)

        combined_dates.append(search_date)

    # return combined_dates,combined_headlines
    return combined_dates,combined_headlines


# print(len(gather_news(stock_name)))
# print(gather_news(stock_name))
list_1=[]
list_2=[]
for single_date in daterange:
    temp_dates,temp_headlines = gather_news(stock_name,normalize_date(single_date.date()))
    list_1.extend(temp_dates)
    list_2.extend(temp_headlines)
    # print(temp_dates)
    # print(temp_headlines)


df = pandas.DataFrame(data={"date": list_1, "headlines": list_2})
df.to_csv("./newsbydate.csv", sep=',',index=False)

In [ ]:
import nltk
nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
pos_words=set(opinion_lexicon.positive())
neg_words=set(opinion_lexicon.negative())

# print(pos_list)

In [ ]:
# from nltk.tokenize.treebank import TreebankWordTokenizer
# tokenizer = TreebankWordTokenizer()

# def sentiment_score(headline):
#     score = 0
#     for word in headline:
#         if word in pos_words:
#             score += 1
#         elif word in neg_words:
#             score -= 1
#     return score

# # This approach does not produce meaningful data that could be used. Therefore, an alternative method should be used. Upon reading Sohangir(2018), it is stated that Vader produced the highest score for Financial Sentiment Lexicon Analysis. Therefore, an alternative method using Vader should be considered.

# for headline in gather_news(stock_name):
#     tokenized_sentence = tokenizer.tokenize(headline)
#     # print(tokenized_sentence)
#     score = sentiment_score(tokenized_sentence)
#     print(score)

In [ ]:
# # Prints individual headlines and their scores by date

# df = pd.read_csv('file.csv')

# del compound_scores
# compound_scores = []

# # filter the dataframe by date
# for single_date in daterange:
#     # temp_date = []
#     norm_date = normalize_date(single_date.date())
#     print('\n')
#     print(norm_date)
#     for headline in (df[(df['date'] == str(norm_date))]).headlines:
#         print(headline)
#         scores = SentimentIntensityAnalyzer().polarity_scores(str(headline))
#         for key in sorted(scores):
#                 print('{0}: {1}, '.format(key, scores[key]), end='\n')
#                 if(key == 'compound' and scores[key]!= 0.0):
#                     compound_scores.append(scores[key])


In [ ]:
df = pd.read_csv('newsbydate.csv')

del compound_scores
compound_scores = []

score_date = []
average_of_date = []
# daterange = pd.date_range('08/10/2020', normalize_date(now))

# filter the dataframe by date
for single_date in daterange:
    norm_date = normalize_date(single_date.date())
    score_date.append(norm_date)
    # print('\n')
    # print(norm_date)
    for headline in (df[(df['date'] == str(norm_date))]).headlines:
        # print(headline)
        scores = SentimentIntensityAnalyzer().polarity_scores(str(headline))
        for key in sorted(scores):
                # print('{0}: {1}, '.format(key, scores[key]), end='\n')
                if(key == 'compound' and scores[key]!= 0.0):
                    compound_scores.append(scores[key])

                    # temp_date.append(scores[key])
    if compound_scores:
        # print(norm_date)
        average_score = (sum(compound_scores)/len(compound_scores))
        average_of_date.append(average_score)
        # print(average_score)
        
df = pandas.DataFrame(data={"date": score_date, "average_score": average_of_date})
df.to_csv("./averagescores.csv", sep=',',index=False)

In [488]:
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as plt
from datetime import datetime as dt


dataframe = web.DataReader(stock_id, 'naver', start='2020-07-01', end=dt.now()).reset_index()
dataframe['Date'] = pd.to_datetime(dataframe['Date']).dt.date



to_int = ['Open','High','Low','Close','Volume']

dataframe[to_int] = dataframe[to_int].astype(str).astype(int)

# Change is defined as the difference in the closing prices of neighbouring dates
dataframe['Change'] = dataframe.Close.diff().fillna(0)
dataframe = dataframe.iloc[:]
dataframe.plot(x='Date',y='Change',figsize=(15,15))

# resample dataframe to include missing weekend/holiday dates and fill the values with 0
dataframe.set_index(dataframe.Date, inplace=True)
dataframe.index = pd.to_datetime(dataframe.index)
dataframe = dataframe.resample('D').sum().fillna(method='ffill')

sentiment = pd.read_csv('averagescores.csv', parse_dates=['date'])
sentiment.index = sentiment.date
sentiment = sentiment.drop('date',axis=1)
sentiment.plot(x='date',y='average_score',figsize=(15,15))

In [500]:
# match starting and ending dates
sentiment.iloc[1:]
dataframe.iloc[:-1]

# add change column to sentiment df
sentiment['change'] = dataframe['Change']
sentiment.replace(0, np.nan, inplace=True)
sentiment = sentiment.fillna(method='ffill')
sentiment = sentiment.fillna(method='bfill')

# sentiment

from sklearn.model_selection import train_test_split

# score increases in a predictable pattern as test_size decreases indicating overfitting.

X_train, X_test, y_train, y_test = train_test_split(sentiment.average_score,sentiment.change,test_size=0.2, random_state=1)


In [501]:
from sklearn.svm import LinearSVC

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# Ys = scaler.fit_transform(pd.DataFrame(Y))


X_train = X_train.values.reshape(-1,1)
y_train = y_train.values.reshape(-1,1)

X_test = X_test.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)

X_train = np.nan_to_num(X_train)
y_train = np.nan_to_num(y_train)

svc = LinearSVC(random_state=1)
svc.fit(X_train, y_train)
print(f"Accuracy: {svc.score(X_test, y_test) * 100:.3f}%", )

Accuracy: 22.222%
